In [99]:
from sklearn.model_selection import train_test_split
from sklearn import svm, metrics
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

In [100]:
def read_data(filepath):
    df = pd.read_csv(filepath, index_col=0)
    return df


In [101]:
filepath_std_1600 = "../database/features/1600_norm.csv"
filepath_std_1600_nfft_2048 = "../database/features/1600_2048_nfft_norm.csv"
filepath_std_1800_nfft_2048_40_mfcc = "../database/features/1600_2048_nfft_norm_40_mfcc.csv"

target_dict = {'happy': 0, 'angry': 1, 'sad': 2, 'relaxed': 3}

In [102]:
df = read_data(filepath_std_1800_nfft_2048_40_mfcc)
df.replace({"emotion": target_dict}, inplace=True)

# columns: 
cent_mean,cent_std,cent_skew,rolloff_mean,rolloff_std,rolloff_skew,flux_mean,flux_std,flux_skew,
flatness_mean,flatness_std,flatness_skew,
mfccs_mean_0,mfccs_mean_1,mfccs_mean_2,mfccs_mean_3,mfccs_mean_4,mfccs_mean_5,mfccs_mean_6,mfccs_mean_7,mfccs_mean_8,
mfccs_mean_9,mfccs_mean_10,mfccs_mean_11,mfccs_mean_12,mfccs_mean_13,mfccs_mean_14,mfccs_mean_15,mfccs_mean_16,
mfccs_mean_17,mfccs_mean_18,mfccs_mean_19,mfccs_std_0,mfccs_std_1,mfccs_std_2,mfccs_std_3,mfccs_std_4,mfccs_std_5,
mfccs_std_6,mfccs_std_7,mfccs_std_8,mfccs_std_9,mfccs_std_10,mfccs_std_11,mfccs_std_12,mfccs_std_13,mfccs_std_14,
mfccs_std_15,mfccs_std_16,mfccs_std_17,mfccs_std_18,mfccs_std_19,contrast_mean_0,contrast_mean_1,contrast_mean_2,
contrast_mean_3,contrast_mean_4,contrast_mean_5,contrast_mean_6,contrast_std_0,contrast_std_1,contrast_std_2,contrast_std_3,
contrast_std_4,contrast_std_5,contrast_std_6,#chroma_mean_0,chroma_mean_1,chroma_mean_2,chroma_mean_3,chroma_mean_4,
chroma_mean_5,chroma_mean_6,chroma_mean_7,chroma_mean_8,chroma_mean_9,chroma_mean_10,chroma_mean_11,chroma_std_0,
chroma_std_1,chroma_std_2,chroma_std_3,chroma_std_4,chroma_std_5,chroma_std_6,chroma_std_7,chroma_std_8,chroma_std_9,
chroma_std_10,chroma_std_11,emotion

As stated in the article best results give stacked MFCC & Spec. Contrast features

In [79]:
for i, col in enumerate(df.columns):
    print(i, col)

0 zcr_mean
1 zcr_std
2 tempo
3 cent_mean
4 cent_std
5 rolloff_mean
6 rolloff_std
7 flux_mean
8 flux_std
9 flatness_mean
10 flatness_std
11 mfccs_mean_0
12 mfccs_mean_1
13 mfccs_mean_2
14 mfccs_mean_3
15 mfccs_mean_4
16 mfccs_mean_5
17 mfccs_mean_6
18 mfccs_mean_7
19 mfccs_mean_8
20 mfccs_mean_9
21 mfccs_mean_10
22 mfccs_mean_11
23 mfccs_mean_12
24 mfccs_mean_13
25 mfccs_mean_14
26 mfccs_mean_15
27 mfccs_mean_16
28 mfccs_mean_17
29 mfccs_mean_18
30 mfccs_mean_19
31 mfccs_mean_20
32 mfccs_mean_21
33 mfccs_mean_22
34 mfccs_mean_23
35 mfccs_mean_24
36 mfccs_mean_25
37 mfccs_mean_26
38 mfccs_mean_27
39 mfccs_mean_28
40 mfccs_mean_29
41 mfccs_mean_30
42 mfccs_mean_31
43 mfccs_mean_32
44 mfccs_mean_33
45 mfccs_mean_34
46 mfccs_mean_35
47 mfccs_mean_36
48 mfccs_mean_37
49 mfccs_mean_38
50 mfccs_mean_39
51 mfccs_std_0
52 mfccs_std_1
53 mfccs_std_2
54 mfccs_std_3
55 mfccs_std_4
56 mfccs_std_5
57 mfccs_std_6
58 mfccs_std_7
59 mfccs_std_8
60 mfccs_std_9
61 mfccs_std_10
62 mfccs_std_11
63 mfccs_std

#### Regular:
From 12 till 51 - mfcc features (12-31 mean, 32-51 std)

From 52 till 65 - contrast features (52-58 mean, 59-65)

#### Additional mfcc:
From 12 till 89 - mfcc features (12-37 mean and std for 22ms, 38-63 mean and std for 1s, 64-80 mean and std for 30s)

From 90 till 103 - contrast features (90-96 mean, 97-103 std)

In [103]:
df_all_X = df.iloc[:, :-1]

df_X_MFCC_SSD_OSC_chroma = df_all_X.iloc[:, 3:127]

df_X_MFCC_SSD = df_all_X.iloc[:, 3:89]

df_X_MFCC_contrast = df_all_X.iloc[:, 11:103]

df_X_MFCC_24ms_contrast = pd.concat([df_all_X.iloc[:, 11:37], df_all_X.iloc[:, 89:103]], axis=1)

df_X_MFCC_24ms_SSD = df_all_X.iloc[:, 3:37]

df_X_MFCC_1s_SSD = pd.concat([df_all_X.iloc[:, 3:11], df_all_X.iloc[:, 37:63]], axis=1)

df_X_MFCC_1s_30s_SSD = pd.concat([df_all_X.iloc[:, 3:11], df_all_X.iloc[:, 37:89]], axis=1)

df_X_MFCC_mean = pd.concat([df_all_X.iloc[:, 11:24], df_all_X.iloc[:, 37:50], df_all_X.iloc[:, 63:76]], axis=1)


to_check = [df_all_X, df_X_MFCC_SSD_OSC_chroma, df_X_MFCC_SSD, df_X_MFCC_contrast, df_X_MFCC_24ms_contrast, df_X_MFCC_24ms_SSD,
           df_X_MFCC_1s_SSD, df_X_MFCC_1s_30s_SSD, df_X_MFCC_mean]

In [104]:
df_all_X

,zcr_mean,zcr_std,tempo,cent_mean,cent_std,rolloff_mean,rolloff_std,flux_mean,flux_std,flatness_mean,...,chroma_std_2,chroma_std_3,chroma_std_4,chroma_std_5,chroma_std_6,chroma_std_7,chroma_std_8,chroma_std_9,chroma_std_10,chroma_std_11
0,0.153988,2.257895,-1.371046,0.597746,1.772348,0.653876,0.768698,3.157145,4.422054,-0.081507,...,-0.968120,-0.224776,-0.822860,-0.481731,-0.907508,-0.774381,0.205492,-0.676236,0.733357,0.111129
1,0.653677,1.238188,-0.436511,1.003175,1.469928,1.132284,1.287805,0.448556,0.761130,0.214220,...,-1.375203,-0.656248,1.440598,-0.059063,1.764701,-0.536086,1.420140,-0.396557,-1.324814,1.096706
2,-0.008601,-0.066854,1.566554,-0.453602,-0.381670,-0.538535,-0.449078,0.285487,0.546296,-0.177957,...,1.626399,0.153019,1.507147,0.817460,0.333133,0.246070,0.634318,0.542814,0.467056,0.977732
3,0.526596,-0.784290,0.902147,0.164090,-1.371665,0.044697,-1.464491,-0.364331,-0.847205,-0.132250,...,-1.594335,0.557407,-0.342345,-0.613533,0.249560,-0.768349,0.562671,-0.459614,-0.081569,0.366973
4,0.683497,0.125210,1.111312,1.257106,0.022003,1.375094,-0.173591,0.218774,0.113455,0.406500,...,-0.013497,-0.301717,0.155355,0.071080,-0.230804,0.545054,-0.210304,0.021681,-0.051181,0.196982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1796,-0.890145,-0.981126,-0.823467,-0.574621,-0.413563,-0.448653,-0.280315,-0.696647,-0.117164,-0.094633,...,0.100799,-1.073068,-0.345143,-0.427609,-0.728821,-0.311437,-0.056555,0.348893,-0.474298,-0.357129
1797,-0.857270,0.336197,-0.436511,-0.863330,0.712962,-0.657919,0.841302,0.423144,0.504498,-0.208865,...,-0.109141,1.620658,0.369993,0.671353,-0.432498,-0.810810,0.430544,0.498105,0.126246,-0.248256
1798,0.481624,-0.382136,0.005724,0.120334,-0.671865,0.042537,-0.665071,-0.183375,-1.126700,-0.135529,...,-0.191753,0.461513,-0.234119,-0.260727,0.889432,0.301179,0.852205,0.820122,0.380649,1.741801
1799,0.138428,1.340668,1.814868,0.247426,0.782353,0.375927,0.052197,-0.737165,-0.631020,-0.091928,...,-0.337249,-0.742737,-0.169752,-0.892254,-1.067843,-0.560721,0.011672,-0.584931,-0.903342,0.036488


In [105]:
df_X_MFCC_24ms = df_all_X.iloc[:, 11:251]
df_X_MFCC_24ms

,mfccs_mean_0,mfccs_mean_1,mfccs_mean_2,mfccs_mean_3,mfccs_mean_4,mfccs_mean_5,mfccs_mean_6,mfccs_mean_7,mfccs_mean_8,mfccs_mean_9,...,mfccs_std_30_30,mfccs_std_30_31,mfccs_std_30_32,mfccs_std_30_33,mfccs_std_30_34,mfccs_std_30_35,mfccs_std_30_36,mfccs_std_30_37,mfccs_std_30_38,mfccs_std_30_39
0,-1.107133,-1.739323,0.674478,0.828749,0.606919,0.929696,1.577304,-0.059638,0.426861,0.367596,...,0.183465,-0.832814,0.337789,-0.224933,-0.946082,-0.613710,-0.487087,0.884297,-0.465025,-0.876117
1,0.206866,-0.794363,0.446715,0.579115,-0.626604,0.095321,-1.138325,0.142541,-1.113675,0.880848,...,-0.161029,-0.822598,-0.427256,-1.076123,-1.087164,-0.481476,-0.120693,-0.338951,-0.690133,-0.263413
2,-0.282394,0.268896,-0.643479,0.339509,1.124886,-0.379753,0.306398,-1.068663,1.018212,-0.892174,...,0.275088,-0.879001,-0.482705,-0.719252,1.746003,1.123743,-0.806022,-0.127461,-0.818138,0.184988
3,0.523192,0.176717,-1.283253,0.706773,-0.464061,0.426325,0.236506,0.124594,0.405789,0.268383,...,1.612297,-0.689571,-0.368555,-0.041151,-0.167737,-0.755859,1.133963,-0.835272,-0.143114,-0.741122
4,0.868375,-0.955947,-0.111723,0.406489,-0.643394,0.946795,-1.052698,1.650697,-0.812321,1.732313,...,1.753539,-0.961903,1.167239,-0.075106,0.360458,0.838589,-0.199033,-0.694842,-1.100323,-0.568066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1796,0.620668,0.564212,-0.028111,0.021477,0.255016,0.074705,0.530990,0.045893,-0.524950,0.050851,...,0.108771,0.931525,0.334809,0.154553,0.224766,0.308928,-0.420861,1.381600,1.154476,-0.311793
1797,-1.448777,0.388228,0.797389,0.304678,0.372561,-0.374384,1.570218,-1.732786,3.038506,-1.448392,...,-1.000881,-0.875508,-1.154259,-0.463436,1.599184,2.775889,2.240483,0.757965,-0.675094,-0.421123
1798,0.503105,0.511182,-1.090800,-0.142475,-1.113140,0.104775,-0.345156,-1.195655,-0.983194,-1.024386,...,0.439536,0.886355,2.297931,-0.525145,0.069384,0.578566,0.478927,0.352328,0.685044,0.650220
1799,-1.172664,-0.501726,-0.198984,0.781069,-0.802022,0.277564,1.290295,-0.021081,0.087119,0.118196,...,-0.353883,-0.370549,0.552855,1.445249,2.673183,0.800488,-0.578398,0.592373,1.256887,1.983324


In [106]:
df_all_Y = df.iloc[:,-1]

In [107]:
df_all_Y

0       0
1       1
2       2
3       0
4       1
       ..
1796    0
1797    1
1798    3
1799    0
1800    2
Name: emotion, Length: 1801, dtype: int64

In [108]:
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(df_X_MFCC_24ms, df_all_Y, test_size=0.3,random_state=45)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df_all_X, df_all_Y, test_size=0.3,random_state=45)

In [91]:
X_train

,zcr_mean,zcr_std,tempo,cent_mean,cent_std,rolloff_mean,rolloff_std,flux_mean,flux_std,flatness_mean,...,chroma_std_2,chroma_std_3,chroma_std_4,chroma_std_5,chroma_std_6,chroma_std_7,chroma_std_8,chroma_std_9,chroma_std_10,chroma_std_11
184,-0.789991,-0.935363,0.003978,-0.680489,-0.384469,-0.557791,-0.429711,-0.304604,0.318314,-0.131381,...,-1.174223,-0.286493,-0.668999,1.578790,-0.006238,-0.592058,0.723027,0.492993,1.341406,-0.769406
934,0.203886,1.740778,2.983048,0.072263,1.907128,-0.063978,1.413343,2.340050,1.747467,-0.063995,...,0.972681,0.602756,1.478967,0.622705,0.223744,0.211944,0.772664,1.336225,1.165408,0.956227
1308,0.486374,3.053502,0.166066,1.127371,1.977859,1.221844,0.214262,0.953543,0.682751,-0.169363,...,-0.109183,-0.105259,-1.029988,-0.802234,-1.142522,-1.132675,-1.655960,-1.635228,-1.635254,-0.867915
1099,-0.251163,-0.035803,-0.573902,-0.185740,-0.062750,-0.100407,-0.043711,0.640835,0.136344,-0.093890,...,-0.686204,0.854788,-0.360095,0.866361,0.067597,-0.808157,1.302478,-0.681005,0.445139,-1.036138
975,-1.085452,-0.621939,-0.298096,-0.942785,0.277889,-0.762698,0.553887,0.625286,1.586800,-0.199548,...,-0.646249,-0.222671,0.379016,-0.371836,0.269876,-0.261963,0.588985,-0.080779,-0.235105,0.274472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,0.415507,0.114228,2.080734,0.693248,0.761399,0.856869,1.020457,-0.745328,0.020876,0.076121,...,0.319161,-1.493821,0.676213,-0.707197,1.317714,-0.273348,-0.215732,0.400000,0.096356,-0.032305
607,-0.932896,-0.373740,-1.274026,-0.623067,-0.240240,-0.515371,-0.491548,0.150776,0.840911,-0.130896,...,-0.411224,-0.429563,0.076552,-0.450052,-0.158363,-0.445645,0.031739,-0.784721,-0.756084,0.363756
1568,0.356333,-1.085035,0.703517,-0.220026,-1.425575,-0.519575,-1.403155,-0.610582,-1.098493,-0.103877,...,-0.508556,-0.152714,-0.216898,-1.052394,-0.668791,-0.537807,0.042638,-0.625225,-0.880100,-0.578633
414,1.617839,-0.876957,0.166066,1.042626,-1.395047,0.597272,-1.329660,-0.206983,-0.752169,0.086097,...,0.168958,-1.083555,-0.329599,0.459306,-1.123099,-0.052408,-0.544200,0.458373,-0.954392,-0.804770


In [109]:
X_train_new

,mfccs_mean_0,mfccs_mean_1,mfccs_mean_2,mfccs_mean_3,mfccs_mean_4,mfccs_mean_5,mfccs_mean_6,mfccs_mean_7,mfccs_mean_8,mfccs_mean_9,...,mfccs_std_30_30,mfccs_std_30_31,mfccs_std_30_32,mfccs_std_30_33,mfccs_std_30_34,mfccs_std_30_35,mfccs_std_30_36,mfccs_std_30_37,mfccs_std_30_38,mfccs_std_30_39
179,0.969067,-0.911462,0.264556,0.449525,-0.711809,1.384383,-0.254402,0.443415,-1.322214,1.281989,...,-0.138436,-0.719470,-0.454956,-0.929828,-0.139825,-0.796160,-0.759217,-0.097286,-0.368651,-1.029293
810,1.289303,-0.651093,-0.730189,0.771798,-0.263762,0.517853,-0.270601,0.945148,0.068421,0.987795,...,0.738963,-0.518822,-0.314977,0.568182,-0.806398,-0.363781,-0.481901,0.028180,0.498421,0.063668
1694,1.011286,0.138703,-0.554984,0.146610,-0.470049,0.781819,-0.040556,0.309064,-0.217421,0.162413,...,-0.079433,-1.109457,0.473398,1.231518,-0.848506,-0.705355,1.697028,-0.240360,-0.729220,-0.758800
1511,-2.040514,2.071907,1.791233,-1.829790,1.416818,-1.001896,0.085378,-0.940529,1.030777,-1.645690,...,0.408726,0.737383,1.739184,-0.529303,-0.412219,1.172996,-0.270390,1.466749,4.657107,1.775355
1472,1.426747,0.154393,-0.886581,0.352075,-0.893409,1.001544,-0.106239,0.405195,-0.334004,0.234856,...,-0.283492,-0.872800,-0.618436,-0.793518,-1.074761,-1.086010,-0.883207,-0.667293,-0.527459,0.113222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,-0.411123,-0.515435,-0.042027,1.013369,2.360689,-0.280943,0.708943,-0.188144,1.854774,-0.017148,...,-0.851886,-0.247763,-1.143447,-1.105927,-0.571979,-0.984880,-0.687484,-0.525525,-0.999424,-0.177978
1568,0.599950,-1.079591,-0.976275,0.873225,-0.028308,0.984301,-0.501960,1.745663,-1.134440,1.160654,...,-0.541218,-0.589309,-1.023274,-1.016447,-1.065106,-1.106780,-0.291095,-0.358347,-0.355993,0.015659
1667,0.624034,-0.669203,0.327224,-0.072577,-1.288988,1.299929,-1.135758,0.510659,-0.728335,0.540000,...,-0.372237,0.825330,-1.086107,-0.845014,-0.675035,-0.175472,-0.527204,-0.460659,-0.226069,-0.763244
414,1.537672,-0.889454,-1.399998,0.948348,-0.471391,-0.012584,-0.619545,0.670726,-0.566498,0.932011,...,-1.025274,0.989927,-0.126967,-0.621785,0.396160,-0.770281,0.426319,-0.041491,-0.669205,-0.342182


In [86]:
print(y_train[y_train == 0].count(),
y_train[y_train == 1].count(),
y_train[y_train == 2].count(),
y_train[y_train == 3].count())

271 272 306 272


## Multiclass SVM

In [44]:
for i in to_check:
    C_range = np.logspace(-2, 10, 13)
    gamma_range = np.logspace(-9, 3, 13)
    param_grid = dict(gamma=gamma_range, C=C_range)
    cv = StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state=42)
    grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
    grid.fit(i, df_all_Y)

    print(
        "The best parameters are %s with a score of %0.2f"
        % (grid.best_params_, grid.best_score_)
    )

The best parameters are {'C': 10000000.0, 'gamma': 1e-06} with a score of 0.27
The best parameters are {'C': 10.0, 'gamma': 1e-05} with a score of 0.26
The best parameters are {'C': 10000000000.0, 'gamma': 1e-09} with a score of 0.26
The best parameters are {'C': 100000.0, 'gamma': 1e-09} with a score of 0.25
The best parameters are {'C': 0.01, 'gamma': 1e-09} with a score of 0.25
The best parameters are {'C': 10000000.0, 'gamma': 1e-07} with a score of 0.26
The best parameters are {'C': 1.0, 'gamma': 0.1} with a score of 0.26
The best parameters are {'C': 10000000000.0, 'gamma': 1e-09} with a score of 0.26
The best parameters are {'C': 100000.0, 'gamma': 1e-08} with a score of 0.26


In [110]:
x_train_new = X_train_new.to_numpy()
y_train_new = y_train_new.to_numpy()
y_test_new = y_test_new.to_numpy()
x_test_new = X_test_new.to_numpy()

In [112]:
svm_clf = svm.SVC(kernel='rbf',gamma=0.000000001, C=1000000000.0, decision_function_shape="ovr", class_weight="balanced")
#svm_clf = svm.SVC(kernel='poly',degree=3, C=0.1, decision_function_shape="ovr", class_weight="balanced")
svm_clf.fit(x_train_new, y_train_new)

y_pred = svm_clf.predict(x_test_new)

print("Accuracy:",metrics.accuracy_score(y_test_new, y_pred))

Accuracy: 0.24399260628465805


Accuracy for std, 1202 records, kernel='rbf',gamma=0.0000001, C=10000000000, decision_function_shape="ovr": 27.57%

Accuracy for std, 1402 records, the best parameters are {'C': 1000.0, 'gamma': 1e-06} with a score of 0.28, just mfcc and contrast

------------------------------------------------------------------------------------
##### For 30s fragments:
* All params - acc 0.27 on test (0.26 whole), 1602 records, kernel='rbf',gamma=0.000000001, C=1000000000.0, decision_function_shape="ovr", nfft=1024
* All params - acc 0.25 on test (? whole), 1602 records, kernel='rbf',gamma=0.000000001, C=1000000000.0, decision_function_shape="ovr", nfft=2048

For consequently df_all_X, df_X_MFCC_SSD_OSC_chroma, df_X_MFCC_SSD, df_X_MFCC_contrast, df_X_MFCC_24ms_contrast, df_X_MFCC_24ms_SSD,df_X_MFCC_1s_SSD, df_X_MFCC_1s_30s_SSD, df_X_MFCC_mean

* The best parameters are {'C': 10000000.0, 'gamma': 1e-06} with a score of 0.27
* The best parameters are {'C': 10.0, 'gamma': 1e-05} with a score of 0.26
* The best parameters are {'C': 10000000000.0, 'gamma': 1e-09} with a score of 0.26
* The best parameters are {'C': 100000.0, 'gamma': 1e-09} with a score of 0.25
* The best parameters are {'C': 0.01, 'gamma': 1e-09} with a score of 0.25
* The best parameters are {'C': 10000000.0, 'gamma': 1e-07} with a score of 0.26
* The best parameters are {'C': 1.0, 'gamma': 0.1} with a score of 0.26
* The best parameters are {'C': 10000000000.0, 'gamma': 1e-09} with a score of 0.26
* The best parameters are {'C': 100000.0, 'gamma': 1e-08} with a score of 0.26
